In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint


In [18]:
DATASET_PATH = 'fer2013.csv'
IMG_SIZE = 48
NUM_CLASSES = 7
BATCH_SIZE = 64
VGG_MODEL_SAVE_PATH = 'vgg16_emotion_model.h5'

In [11]:
def load_and_preprocess_data(df):
    
    print(f"Loading and processing {len(df)} samples...")
    
    
    X_gs = np.array([
        np.fromstring(row, dtype=int, sep=' ').reshape(IMG_SIZE, IMG_SIZE) 
        for row in df['pixels']
    ], dtype='float32')

    
    X_gs = np.expand_dims(X_gs, axis=-1)

    
    X_rgb = np.repeat(X_gs, 3, axis=-1)
    
    
    X_rgb /= 255.0

    
    y = to_categorical(df['emotion'], num_classes=NUM_CLASSES)
    
    print(f"VGG16 Input Shape: {X_rgb.shape}")
    return X_rgb, y

In [13]:
def build_vgg16_model(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    
    
    vgg_base = VGG16(
        weights='imagenet',       
        include_top=False,        
        input_shape=input_shape 
    )

    
    for layer in vgg_base.layers:
        layer.trainable = False

    
    model = Sequential([
        vgg_base,                   
        Flatten(),                  
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation='softmax') 
    ])

    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    
    print("\n--- VGG16 Transfer Model Architecture ---")
    model.summary()
    return model

In [20]:
if __name__ == '__main__':
    
    try:
        data = pd.read_csv(DATASET_PATH)
    except FileNotFoundError:
        print(f"ERROR: The file '{DATASET_PATH}' was not found. Please verify the path.")
        exit()

    
    train_data = data[data['Usage'] == 'Training']
    test_data = data[data['Usage'] == 'PublicTest']

   
    X_train_vgg, y_train = load_and_preprocess_data(train_data)
    X_test_vgg, y_test = load_and_preprocess_data(test_data)
    
    
    vgg_model = build_vgg16_model()

    
    train_datagen = ImageDataGenerator(
        rotation_range=10, 
        zoom_range=0.1,
        width_shift_range=0.1, 
        height_shift_range=0.1, 
        fill_mode='nearest'
    )
    train_generator = train_datagen.flow(X_train_vgg, y_train, batch_size=BATCH_SIZE)

    
    callbacks = [
        ModelCheckpoint(VGG_MODEL_SAVE_PATH, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
    ]

    EPOCHS = 150
    print("\n--- Starting VGG16 Transfer Learning Training with Augmentation ---")
    
    vgg_model.fit(
        train_generator,
        steps_per_epoch=len(X_train_vgg) // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_test_vgg, y_test),
        callbacks=callbacks,
        verbose=1
    )

    
    vgg_model.load_weights(VGG_MODEL_SAVE_PATH)
    
    print("\n--- Final VGG16 Model Evaluation (Testing) ---")
    loss, accuracy = vgg_model.evaluate(X_test_vgg, y_test, verbose=0)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy*100:.2f}%")

Loading and processing 28709 samples...
VGG16 Input Shape: (28709, 48, 48, 3)
Loading and processing 3589 samples...
VGG16 Input Shape: (3589, 48, 48, 3)

--- VGG16 Transfer Model Architecture ---


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,982,983 (57.16 MB)

 Trainable params: 267,271 (1.02 MB)

...

448/448 ━━━━━━━━━━━━━━━━━━━━ 27s 60ms/step - accuracy: 0.4375 - loss: 1.3780 - val_accuracy: 0.4224 - val_loss: 1.5393
Epoch 63/150
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step - accuracy: 0.3992 - loss: 1.5449
Epoch 63: val_accuracy did not improve from 0.42240
448/448 ━━━━━━━━━━━━━━━━━━━━ 216s 482ms/step - accuracy: 0.3964 - loss: 1.5475 - val_accuracy: 0.4213 - val_loss: 1.5222
Epoch 64/150
  1/448 ━━━━━━━━━━━━━━━━━━━━ 3:05 415ms/step - accuracy: 0.5781 - loss: 1.3567
Epoch 64: val_accuracy improved from 0.42240 to 0.42296, saving model to vgg16_emotion_model.h5


448/448 ━━━━━━━━━━━━━━━━━━━━ 27s 60ms/step - accuracy: 0.5781 - loss: 1.3567 - val_accuracy: 0.4230 - val_loss: 1.5223
Epoch 65/150
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step - accuracy: 0.3976 - loss: 1.5441
Epoch 65: val_accuracy did not improve from 0.42296
448/448 ━━━━━━━━━━━━━━━━━━━━ 220s 491ms/step - accuracy: 0.3986 - loss: 1.5452 - val_accuracy: 0.4126 - val_loss: 1.5277
Epoch 66/150
  1/448 ━━━━━━━━━━━━━━━━━━━━ 3:37 486ms/step - accuracy: 0.4219 - loss: 1.4629
Epoch 66: val_accuracy did not improve from 0.42296
448/448 ━━━━━━━━━━━━━━━━━━━━ 25s 54ms/step - accuracy: 0.4219 - loss: 1.4629 - val_accuracy: 0.4132 - val_loss: 1.5281
Epoch 67/150
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step - accuracy: 0.3984 - loss: 1.5441
Epoch 67: val_accuracy did not improve from 0.42296
448/448 ━━━━━━━━━━━━━━━━━━━━ 218s 486ms/step - accuracy: 0.3977 - loss: 1.5447 - val_accuracy: 0.4121 - val_loss: 1.5255
Epoch 68/150
  1/448 ━━━━━━━━━━━━━━━━━━━━ 3:09 424ms/step - accuracy: 0.4219 - loss: 1.4426


448/448 ━━━━━━━━━━━━━━━━━━━━ 221s 493ms/step - accuracy: 0.4008 - loss: 1.5410 - val_accuracy: 0.4260 - val_loss: 1.5102
Epoch 80/150
  1/448 ━━━━━━━━━━━━━━━━━━━━ 2:58 399ms/step - accuracy: 0.3906 - loss: 1.4815
Epoch 80: val_accuracy did not improve from 0.42602
448/448 ━━━━━━━━━━━━━━━━━━━━ 25s 54ms/step - accuracy: 0.3906 - loss: 1.4815 - val_accuracy: 0.4255 - val_loss: 1.5098
Epoch 81/150
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step - accuracy: 0.3998 - loss: 1.5383
Epoch 81: val_accuracy did not improve from 0.42602
448/448 ━━━━━━━━━━━━━━━━━━━━ 217s 485ms/step - accuracy: 0.3988 - loss: 1.5418 - val_accuracy: 0.4213 - val_loss: 1.5096
Epoch 82/150
  1/448 ━━━━━━━━━━━━━━━━━━━━ 3:07 418ms/step - accuracy: 0.4062 - loss: 1.4411
Epoch 82: val_accuracy did not improve from 0.42602
448/448 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.4062 - loss: 1.4411 - val_accuracy: 0.4213 - val_loss: 1.5102
Epoch 83/150
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step - accuracy: 0.4042 - loss: 1.5302


448/448 ━━━━━━━━━━━━━━━━━━━━ 222s 496ms/step - accuracy: 0.4053 - loss: 1.5343 - val_accuracy: 0.4296 - val_loss: 1.5083
Epoch 100/150
  1/448 ━━━━━━━━━━━━━━━━━━━━ 3:01 406ms/step - accuracy: 0.4375 - loss: 1.6554
Epoch 100: val_accuracy did not improve from 0.42965
448/448 ━━━━━━━━━━━━━━━━━━━━ 24s 54ms/step - accuracy: 0.4375 - loss: 1.6554 - val_accuracy: 0.4271 - val_loss: 1.5079
Epoch 101/150
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step - accuracy: 0.4079 - loss: 1.5262
Epoch 101: val_accuracy did not improve from 0.42965
448/448 ━━━━━━━━━━━━━━━━━━━━ 218s 486ms/step - accuracy: 0.4056 - loss: 1.5334 - val_accuracy: 0.4196 - val_loss: 1.5355
Epoch 102/150
  1/448 ━━━━━━━━━━━━━━━━━━━━ 3:07 419ms/step - accuracy: 0.4219 - loss: 1.5404
Epoch 102: val_accuracy did not improve from 0.42965
448/448 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.4219 - loss: 1.5404 - val_accuracy: 0.4210 - val_loss: 1.5357
Epoch 103/150
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step - accuracy: 0.4116 - loss: 

448/448 ━━━━━━━━━━━━━━━━━━━━ 220s 491ms/step - accuracy: 0.4067 - loss: 1.5318 - val_accuracy: 0.4363 - val_loss: 1.4951
Epoch 104/150
  1/448 ━━━━━━━━━━━━━━━━━━━━ 3:14 435ms/step - accuracy: 0.4219 - loss: 1.6244
Epoch 104: val_accuracy did not improve from 0.43633
448/448 ━━━━━━━━━━━━━━━━━━━━ 25s 54ms/step - accuracy: 0.4219 - loss: 1.6244 - val_accuracy: 0.4349 - val_loss: 1.4954
Epoch 105/150
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step - accuracy: 0.4112 - loss: 1.5242
Epoch 105: val_accuracy did not improve from 0.43633
448/448 ━━━━━━━━━━━━━━━━━━━━ 218s 486ms/step - accuracy: 0.4064 - loss: 1.5280 - val_accuracy: 0.4271 - val_loss: 1.5082
Epoch 106/150
  1/448 ━━━━━━━━━━━━━━━━━━━━ 2:57 398ms/step - accuracy: 0.4062 - loss: 1.5697
Epoch 106: val_accuracy did not improve from 0.43633
448/448 ━━━━━━━━━━━━━━━━━━━━ 25s 54ms/step - accuracy: 0.4062 - loss: 1.5697 - val_accuracy: 0.4269 - val_loss: 1.5092
Epoch 107/150
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step - accuracy: 0.4046 - loss: 

In [21]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

vgg_model.load_weights(VGG_MODEL_SAVE_PATH)

print("\n--- Final VGG16 Model Evaluation (Testing) ---")
loss, accuracy = vgg_model.evaluate(X_test_vgg, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")


y_pred_probs = vgg_model.predict(X_test_vgg, verbose=0)

y_pred_labels = np.argmax(y_pred_probs, axis=1)

y_true_labels = np.argmax(y_test, axis=1)

EMOTION_LABELS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

report = classification_report(
    y_true_labels, 
    y_pred_labels, 
    target_names=EMOTION_LABELS
)

print("\n--- Classification Report (Precision, Recall, F1-Score) ---")
print(report)

conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)
print("\n--- Confusion Matrix ---")
print(conf_matrix)


--- Final VGG16 Model Evaluation (Testing) ---
Test Loss: 1.4951
Test Accuracy: 43.63%

--- Classification Report (Precision, Recall, F1-Score) ---
              precision    recall  f1-score   support

       Angry       0.36      0.29      0.32       467
     Disgust       0.33      0.02      0.03        56
        Fear       0.32      0.23      0.27       496
       Happy       0.48      0.67      0.56       895
         Sad       0.42      0.31      0.36       653
    Surprise       0.63      0.58      0.60       415
     Neutral       0.36      0.46      0.41       607

    accuracy                           0.44      3589
   macro avg       0.42      0.36      0.36      3589
weighted avg       0.43      0.44      0.42      3589


--- Confusion Matrix ---
[[136   0  32 129  60  19  91]
 [  8   1  13  16   7   2   9]
 [ 52   0 112 106  78  62  86]
 [ 46   0  46 599  68  24 112]
 [ 63   0  73 171 200  15 131]
 [ 28   1  27  52  11 239  57]
 [ 49   1  45 166  48  19 279]]
